## Conclusion from SQL analysis
### Needed data
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* Can keep recovered audit
### To do
* verify control area is unique
* find duplicate data

In [2]:
from sqlalchemy import create_engine
#from sqlalchemy import inspect
import pandas as pd

In [3]:
engine = create_engine("sqlite:///mta.db")
df_check = pd.read_sql('SELECT * FROM mta_data;', engine)
df_check.head(3)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,03:00:00,REGULAR,7537255,2570434
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,07:00:00,REGULAR,7537260,2570445
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,11:00:00,REGULAR,7537285,2570500


In [19]:
#check duplicate data
(df_check
 .groupby(["CA", "UNIT", "SCP", "STATION", "DATE", "TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(10)

,CA,UNIT,SCP,STATION,DATE,TIME,ENTRIES
248165,B024,R211,00-05-00,KINGS HWY,05/04/2021,12:00:00,2
248180,B024,R211,00-05-00,KINGS HWY,05/07/2021,00:00:00,2
897516,N100,R252,00-00-00,HIGH ST,03/08/2021,15:00:00,2
897515,N100,R252,00-00-00,HIGH ST,03/08/2021,11:00:00,2
2559787,R417,R222,00-03-01,PARKCHESTER,03/26/2021,05:00:00,2
897513,N100,R252,00-00-00,HIGH ST,03/08/2021,03:00:00,2
935814,N114,R297,01-00-01,FRANKLIN AV,03/10/2021,08:00:00,2
897512,N100,R252,00-00-00,HIGH ST,03/07/2021,23:00:00,2
897511,N100,R252,00-00-00,HIGH ST,03/07/2021,19:00:00,2
897510,N100,R252,00-00-00,HIGH ST,03/07/2021,15:00:00,2


In [35]:
#find the duplicate data
dup = (df_check
       .groupby(["CA", "UNIT", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES", ascending=False))
dup[dup.ENTRIES > 1]

,CA,UNIT,SCP,STATION,DATE,TIME,ENTRIES
248165,B024,R211,00-05-00,KINGS HWY,05/04/2021,12:00:00,2
248180,B024,R211,00-05-00,KINGS HWY,05/07/2021,00:00:00,2
897516,N100,R252,00-00-00,HIGH ST,03/08/2021,15:00:00,2
897515,N100,R252,00-00-00,HIGH ST,03/08/2021,11:00:00,2
2559787,R417,R222,00-03-01,PARKCHESTER,03/26/2021,05:00:00,2
...,...,...,...,...,...,...,...
1946564,R138,R293,00-02-04,34 ST-PENN STA,02/28/2021,09:00:00,2
248171,B024,R211,00-05-00,KINGS HWY,05/05/2021,12:00:00,2
793278,N071,R013,00-00-02,34 ST-PENN STA,04/08/2021,08:00:00,2
792098,N071,R013,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,2


In [34]:
#verify CA is unique
dup = (df_check
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES", ascending=False))
dup[dup.ENTRIES > 1]

,CA,SCP,STATION,DATE,TIME,ENTRIES
248165,B024,00-05-00,KINGS HWY,05/04/2021,12:00:00,2
248180,B024,00-05-00,KINGS HWY,05/07/2021,00:00:00,2
897516,N100,00-00-00,HIGH ST,03/08/2021,15:00:00,2
897515,N100,00-00-00,HIGH ST,03/08/2021,11:00:00,2
2559787,R417,00-03-01,PARKCHESTER,03/26/2021,05:00:00,2
...,...,...,...,...,...,...
1946564,R138,00-02-04,34 ST-PENN STA,02/28/2021,09:00:00,2
248171,B024,00-05-00,KINGS HWY,05/05/2021,12:00:00,2
793278,N071,00-00-02,34 ST-PENN STA,04/08/2021,08:00:00,2
792098,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,2


In [21]:
#example1
mask = ((df_check["CA"] == "B024") & 
(df_check["SCP"] == "00-05-00") & 
(df_check["STATION"] == "KINGS HWY") &
(df_check["DATE"] == "05/04/2021") &
(df_check["TIME"] == "12:00:00"))

df_check[mask]

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1899060,B024,R211,00-05-00,KINGS HWY,BQ,BMT,05/04/2021,12:00:00,REGULAR,262152,0
1899061,B024,R211,00-05-00,KINGS HWY,BQ,BMT,05/04/2021,12:00:00,RECOVR AUD,18,0


In [36]:
#example2
#will need to clean duplicate data with both regular and recovered
mask = ((df_check["CA"] == "N071") & 
(df_check["SCP"] == "00-00-00") & 
(df_check["STATION"] == "34 ST-PENN STA") &
(df_check["DATE"] == "04/08/2021") &
(df_check["TIME"] == "08:00:00"))

df_check[mask]

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1100666,N071,R013,00-00-00,34 ST-PENN STA,ACE,IND,04/08/2021,08:00:00,REGULAR,241178,732169
1100667,N071,R013,00-00-00,34 ST-PENN STA,ACE,IND,04/08/2021,08:00:00,RECOVR AUD,6336210,17886963


In [37]:
for col in df_check:
    print(col, df_check[col].nunique())

CA 750
UNIT 469
SCP 224
STATION 379
LINENAME 114
DIVISION 6
DATE 98
TIME 58311
DESC 2
ENTRIES 1907237
EXITS 1808151


In [9]:
pd.read_sql('SELECT * FROM mta_data;', engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927288 entries, 0 to 2927287
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   CA        object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 245.7+ MB


In [4]:
#df = pd.read_sql('SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data;', engine)
df = pd.read_sql('''SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data
                    WHERE date NOT IN ("02/27/2021", "02/28/2021", "06/01/2021", 
                                       "06/02/2021", "06/03/2021", "06/04/2021")
                    ORDER BY ca, date, time, scp;''', engine)

In [5]:
print(df.head(3))
print(df.tail(3))

     CA       SCP STATION        DATE      TIME     DESC  ENTRIES    EXITS
0  A002  02-00-00   59 ST  03/01/2021  03:00:00  REGULAR  7537860  2570718
1  A002  02-00-01   59 ST  03/01/2021  03:00:00  REGULAR  6677242  1516347
2  A002  02-03-00   59 ST  03/01/2021  03:00:00  REGULAR  1416509  5346219
            CA       SCP        STATION        DATE      TIME     DESC  \
2747754  TRAM2  00-03-01  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   
2747755  TRAM2  00-05-00  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   
2747756  TRAM2  00-05-01  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   

         ENTRIES   EXITS  
2747754  3133585  397464  
2747755        0       0  
2747756     5554     577  


In [6]:
#save almost 80 mb
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747757 entries, 0 to 2747756
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   CA       object
 1   SCP      object
 2   STATION  object
 3   DATE     object
 4   TIME     object
 5   DESC     object
 6   ENTRIES  int64 
 7   EXITS    int64 
dtypes: int64(2), object(6)
memory usage: 167.7+ MB


In [13]:
for col in df:
    print(df[col].value_counts())

PTH22    23551
R610     13798
PTH03    13540
PTH02    12173
R238     12130
         ...  
R177      1440
J022      1104
H037      1104
A085       552
OB01       313
Name: CA, Length: 750, dtype: int64
00-00-00    254844
00-00-01    253111
00-00-02    225012
00-00-03    113928
00-03-00     99554
             ...  
00-04-09       521
02-02-04       520
03-01-07       508
03-00-08         3
01-FF-02         1
Name: SCP, Length: 224, dtype: int64
34 ST-PENN STA     56370
FULTON ST          53013
GRD CNTRL-42 ST    49022
23 ST              40300
86 ST              36002
                   ...  
CHAUNCEY ST         1640
HARLEM 148 ST       1640
SUTTER AV           1104
CLEVELAND ST        1104
ORCHARD BEACH        313
Name: STATION, Length: 379, dtype: int64
04/12/2021    30321
03/19/2021    30167
03/16/2021    30147
04/07/2021    30094
03/15/2021    30073
              ...  
04/29/2021    29800
05/23/2021    29789
05/02/2021    29789
05/14/2021    29761
03/14/2021    27156
Name: DATE, Lengt

In [8]:
df[df.CA == "PTH22"] # too slow

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS
1685645,PTH22,00-01-03,PATH NEW WTC,03/01/2021,00:01:28,REGULAR,161823,153273
1685646,PTH22,00-03-07,PATH NEW WTC,03/01/2021,00:04:38,REGULAR,138237,173717
1685647,PTH22,00-01-00,PATH NEW WTC,03/01/2021,00:10:15,REGULAR,260659,50029
1685648,PTH22,00-00-02,PATH NEW WTC,03/01/2021,00:12:20,REGULAR,294638,242269
1685649,PTH22,00-04-05,PATH NEW WTC,03/01/2021,00:16:41,REGULAR,693,1992
...,...,...,...,...,...,...,...,...
1709191,PTH22,00-03-05,PATH NEW WTC,05/31/2021,23:15:59,REGULAR,98553,165289
1709192,PTH22,00-02-04,PATH NEW WTC,05/31/2021,23:16:48,REGULAR,12354,9612
1709193,PTH22,00-03-03,PATH NEW WTC,05/31/2021,23:32:50,REGULAR,274694,367287
1709194,PTH22,00-02-07,PATH NEW WTC,05/31/2021,23:34:27,REGULAR,65655,19646


In [9]:
pd.read_sql('SELECT scp, COUNT(DISTINCT scp) FROM mta_data WHERE ca = "PTH22";', engine) #much faster

,SCP,COUNT(DISTINCT scp)
0,00-00-00,45


In [10]:
pd.read_sql('''SELECT a.ca, a.scp, a.station, a.date, a.time, a.entries, a.exits, b.date, b.time, b.entries, b.exits
               FROM mta_data a, mta_data b 
               WHERE a.ca = b.ca 
                 AND a.scp = b.scp
                 AND a.date = b.date
                 AND b.time > a.time
               LIMIT 20;''', engine)

,CA,SCP,STATION,DATE,TIME,ENTRIES,EXITS,DATE,TIME,ENTRIES,EXITS
0,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,07:00:00,7537260,2570445
1,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,11:00:00,7537285,2570500
2,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,15:00:00,7537354,2570546
3,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,19:00:00,7537501,2570582
4,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,23:00:00,7537589,2570596
5,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,11:00:00,7537285,2570500
6,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,15:00:00,7537354,2570546
7,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,19:00:00,7537501,2570582
8,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,23:00:00,7537589,2570596
9,A002,02-00-00,59 ST,02/27/2021,11:00:00,7537285,2570500,02/27/2021,15:00:00,7537354,2570546


In [11]:
# doesn't work for overnight
pd.read_sql('''SELECT * 
               FROM (SELECT a.ca, a.scp, a.station, a.date, a.time, a.entries, a.exits, b.date, b.time, b.entries, b.exits,
                            ROW_NUMBER() Over (Partition By a.ca, a.scp, a.date, a.time Order By a.time) SORT
                     FROM mta_data a, mta_data b 
                     WHERE a.ca = b.ca 
                       AND a.scp = b.scp
                       AND a.date = b.date
                       AND b.time > a.time
                     LIMIT 20)
               WHERE sort = 1;''', engine)

,ca,scp,station,date,time,entries,exits,date:1,time:1,entries:1,exits:1,SORT
0,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,07:00:00,7537260,2570445,1
1,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,11:00:00,7537285,2570500,1
2,A002,02-00-00,59 ST,02/27/2021,11:00:00,7537285,2570500,02/27/2021,15:00:00,7537354,2570546,1
3,A002,02-00-00,59 ST,02/27/2021,15:00:00,7537354,2570546,02/27/2021,19:00:00,7537501,2570582,1
4,A002,02-00-00,59 ST,02/27/2021,19:00:00,7537501,2570582,02/27/2021,23:00:00,7537589,2570596,1
5,A002,02-00-00,59 ST,02/28/2021,03:00:00,7537598,2570598,02/28/2021,07:00:00,7537602,2570602,1


In [12]:
for col in df:
    print(col, df[col].nunique())

CA 750
SCP 224
STATION 379
DATE 92
TIME 56818
DESC 2
ENTRIES 1806455
EXITS 1717090


## Conclusion from SQL and panda pre analysis
### Needed data
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* get needed columns to save almost 80 mb
* Can keep desc-recovered but will need to clean duplicate data with both regular and recovered
* sql query may be faster